<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning/blob/master/homework_2_graded/Multi_Label_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-label classification
Difference between multi-class and multi-label is that multi-class classification is mutually exclusive, while multi-label classification can assign multiple labels to a given example.

https://towardsdatascience.com/multi-label-image-classification-in-tensorflow-2-0-7d4cf8a4bc72

https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff